In [1]:
import os, json, pandas as pd
from pathlib import Path
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

VECTOR_DIR = "data/vectorstore"

hf_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2")

store = Chroma(
    persist_directory=VECTOR_DIR,
    embedding_function=hf_embeddings,
)
retriever = store.as_retriever(
    search_type="similarity", search_kwargs={"k": 3})

llm = ChatOpenAI(model="gpt-5-nano", temperature=0)

def rag(question: str):
    docs = retriever.invoke(question)
    contexts = [d.page_content for d in docs]
    return {"question": question,
            "contexts": contexts,
            "metadata": [d.metadata for d in docs]}

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful assistant. "
     "답변은 질문에 입력된 언어로 하며, 아래 제공된 contexts에 있는 정보만 사용해. "
     "contexts에 없는 내용은 추정하지 말고 '제공된 자료로는 확인할 수 없습니다'라고 답해."),
    ("human",
     "Question:\n{question}\n\n"
     "Contexts (use ONLY this info):\n{ctx}\n\n")
])

def generate_answer(question: str, contexts: list[str]) -> str:
    if not contexts:
        return "제공된 자료로는 확인할 수 없습니다."
    ctx_joined = "\n".join(f"- {c}" for c in contexts)
    messages = prompt.format_messages(question=question, ctx=ctx_joined)
    resp = llm.invoke(messages)
    return resp.content.strip()

c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\캡스톤디자인\chatbot-api\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import glob

# 모든 scenario_*.jsonl 파일을 찾아서 DATASET 리스트로 만듦
DATASET_PATHS = sorted(glob.glob("data/scenario_*.jsonl"))
OUTDIR  = Path("results")

def load_gold_rows(paths: list[str]):
    rows = []
    for path in paths:
        path = Path(path)
        with path.open("r", encoding="utf-8") as f:
            for line in f:
                if not line.strip():
                    continue
                item = json.loads(line)
                rows.append(item)
    return rows


gold = load_gold_rows(DATASET_PATHS)
print(f"Loaded {len(gold)} questions from {len(DATASET_PATHS)} files.")

# 1) 질문별로 retrieval + generation 실행
preds = []
for item in gold:
    q  = item["question"]
    gt = item["ground_truth"] 
    r  = rag(q)                # {"question","contexts","metadata"}
    ctxs = r.get("contexts", [])
    ans  = generate_answer(q, ctxs)
    preds.append({
        "question": q,
        "answer": ans,
        "contexts": ctxs,
        "ground_truth": gt,
    })

# 2) HuggingFace Dataset 변환
df = pd.DataFrame(preds)
ds = Dataset.from_pandas(df)

# 3) RAGAS 메트릭 계산
result = evaluate(
    ds,
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall],
)

result_df = result.to_pandas()
display(result_df)

import datetime
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
# Calculate mean scores for each metric
result_df['faithfulness_mean'] = result_df['faithfulness'].mean()
result_df['answer_relevancy_mean'] = result_df['answer_relevancy'].mean()
result_df['context_precision_mean'] = result_df['context_precision'].mean()
result_df['context_recall_mean'] = result_df['context_recall'].mean()

csv_filename = f"{current_time}.csv"
csv_path = OUTDIR / csv_filename
result_df.to_csv(csv_path, index=False, encoding='utf-8')
print(f"Results saved to: {csv_path}")

Loaded 16 questions from 2 files.


Evaluating:   3%|▎         | 2/64 [00:19<09:54,  9.59s/it]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating:  62%|██████▎   | 40/64 [03:57<01:48,  4.53s/it]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating: 100%|██████████| 64/64 [06:00<00:00,  5.63s/it]


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,졸업논문 작성은 어떻게 해야돼?,[[학위청구논문 작성요령] 논문의 내용은 이론적으로 중복됨이 없이 간단 명료하게 기...,다음은 제공된 자료를 바탕으로 정리한 졸업논문 작성 방법입니다.\n\n1) 작성 원...,"공주대 대학원 지침에 따라 논문은 간단·명료하게 작성하며, 제목(국·영문), 목차와...",0.955556,0.827981,1.000000,0.666667
1,학위논문 편집·인쇄 요령 핵심만 알려줘.,[[편집·인쇄 및 제출 요령 개요] 학위논문 심사위원회 최종 통과 논문은 심사 완료...,학위논문 편집·인쇄 요령의 핵심 요약:\n\n- 제출 시기: 심사 완료 후 1개월 ...,"심사위원회 최종 통과 후 1개월 이내 인쇄 제출. 판종 4×6배판(19×26cm),...",0.909091,0.822716,0.833333,1.000000
2,제본 순서는 어떻게 돼?,[[학위청구논문 작성요령] 논문의 내용은 이론적으로 중복됨이 없이 간단 명료하게 기...,제본 순서는 다음과 같습니다:\n1) 겉표지: 별지1 규격/양식 \n2) 겉표지 ...,제본 순서는 ①겉표지(별지1) ②백지 1장 ③논문제출지(별지2) ④인준지(석사·박사...,1.000000,0.805364,1.000000,1.000000
3,도서관에 제출할 인쇄본은 몇 부야? 어디에 제출해?,[[학위청구논문 작성요령] 논문의 내용은 이론적으로 중복됨이 없이 간단 명료하게 기...,제공된 자료로는 도서관에 제출할 인쇄본의 부수와 제출 장소를 확인할 수 없습니다.,"인쇄본은 인준날인 원본 1부를 포함하여 총 3부를 제출하며, 제출장소는 중앙도서관 ...",0.000000,0.000000,1.000000,0.000000
4,원문 파일(전자 제출)은 어떻게 해?,[[제본 순서] 1) 겉표지: 별지1 규격/양식. 2) 겉표지 다음 백지 1장. 3...,원문 파일(전자 제출)은 다음 경로로 제출합니다.\n- 도서관 홈페이지 → 연구·학...,도서관 홈페이지(연구·학습지원→학위논문제출) 또는 dCollection에서 온라인으...,0.600000,0.000000,1.000000,1.000000
5,학위논문 공개는 원칙적으로 어떻게 돼? 비공개 가능해?,[[학위청구논문 작성요령] 논문의 내용은 이론적으로 중복됨이 없이 간단 명료하게 기...,원칙적으로 공개됩니다. 다만 비공개를 원할 경우에는 저작권 비동의 사유서를 첨부해야...,"학위논문은 원칙적으로 공개하며, 특별히 비공개할 사유가 있는 경우 학위논문 저작권 ...",1.000000,0.000000,0.500000,1.000000
6,How can I extend my visa in Korea?,[[학위청구논문 작성요령] 논문의 내용은 이론적으로 중복됨이 없이 간단 명료하게 기...,제공된 자료에 따르면 비자 일반 연장 절차에 대한 상세 정보보다는 “수료연장(학적 ...,You can apply from up to 3 months before your ...,0.888889,0.000000,0.333333,0.333333
7,What documents do I need for the extension?,[1. 외국인 등록 ▶ 모든 외국인은 한국 입국 후 90일 이내에 관할 출입국 관리...,There are two extension categories with differ...,For a general extension: ① application ② passp...,1.000000,0.901449,0.500000,1.000000
8,How much is the extension fee?,[4. 체류지 변경 신고 ▶ 주소(체류지) 변경 시 14일 이내에 관할 주민센터 또...,"제공된 자료로는 확인할 수 없습니다.\n\n참고로 외국인 등록 수수료는 35,000...","The extension fee is 60,000 KRW. Government Sc...",0.666667,0.000000,0.000000,0.000000
9,What are the financial requirements for an ext...,[4. 체류지 변경 신고 ▶ 주소(체류지) 변경 시 14일 이내에 관할 주민센터 또...,"- 일반연장(학부생, 대학원생)의 재정 요건: 재정능력입증서류를 제출해야 하며, 1...","For a general extension, you need financial pr...",0.800000,0.937164,0.333333,0.500000


Results saved to: results\20251015_162641.csv
